In [1]:
import requests
from bs4 import BeautifulSoup
from os.path  import basename
import time as t
from random import randrange as rr
import json
import numpy as np
import pandas as pd
import hashlib

C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [3]:
# season_list = [x for x in range(2010, 2019)]
# season_list

In [4]:
# season_list = [x for x in range(2010, 2019)]

# for season in season_list:
#     page = 'https://www.transfermarkt.at/bundesliga/startseite/wettbewerb/A1/plus/?saison_id=' + str(season)

#     # beginn der Zeitmessung und übergabe des html dokuments
#     start = t.time()
#     tree = requests.get(page, headers = headers)
#     soup = BeautifulSoup(tree.content, 'html.parser')
    
#     club_list = []


#     for i in range(len(soup.select('a.vereinprofil_tooltip'))):
#         try:
#             club_list.append(soup.select('a.vereinprofil_tooltip')[i].attrs['id'])
#         except: KeyError

#     club_list = list(set(club_list))
#     print(club_list)
#     t.sleep(3)

In [5]:
# club_list = []


# for i in range(len(soup.select('a.vereinprofil_tooltip'))):
#     try:
#         club_list.append(soup.select('a.vereinprofil_tooltip')[i].attrs['id'])
#     except: KeyError
        
# club_list = list(set(club_list))
# print(club_list)

In [6]:
# page = 'https://www.transfermarkt.at/red-bull-salzburg/spielplan/verein/2662/saison_id/2018/plus/1'


# # beginn der Zeitmessung und übergabe des html dokuments
# start = t.time()
# tree = requests.get(page, headers = headers)
# soup = BeautifulSoup(tree.content, 'html.parser')


In [8]:
game_list = []
season_list = [x for x in range(2009, 2019)]


for season in season_list:
    print(season)
    t.sleep(1)
    page_league = 'https://www.transfermarkt.at/bundesliga/startseite/wettbewerb/A1/plus/?saison_id=' + str(season)

    tree = requests.get(page_league, headers = headers)
    soup = BeautifulSoup(tree.content, 'html.parser')
    
    club_list = []


    for i in range(len(soup.select('a.vereinprofil_tooltip'))):
        try:
            club_list.append(soup.select('a.vereinprofil_tooltip')[i].attrs['id'])
        except: KeyError

    club_list = list(set(club_list))
        

    for club in club_list:
        t.sleep(3)
        page_club = 'https://www.transfermarkt.at/red-bull-salzburg/spielplan/verein/' + str(club) + '/saison_id/' + str(season) + '/plus/1'
        # beginn der Zeitmessung und übergabe des html dokuments
        start = t.time()
        tree = requests.get(page_club, headers = headers)
        soup = BeautifulSoup(tree.content, 'html.parser')


        for i in range(3, len(soup.select('div.box'))):
            box = soup.select('div.box')[i]
            competition = soup.select('div.box')[i]('div')[0]('img')[0].attrs['alt']
            saison = str(season) + '/' + str(int(str(season)[-2:])+1)

            for j in range(1, len(box.select('tr'))):
                line = box.select('tr')[j]

                gameday = line.select('td')[0].get_text().strip()
                date = line.select('td')[1].get_text()
                time = line.select('td')[2].get_text()
                home_team_id = line.select('td')[3]('a')[0].attrs['id']
                home_team_name = line.select('td')[3]('img')[0].attrs['alt']
                guest_team_id = line.select('td')[5]('a')[0].attrs['id']
                guest_team_name = line.select('td')[5]('img')[0].attrs['alt']
                
                if club == home_team_id:
                    home_coach = line.select('td')[8].get_text()
                    guest_coach = ''
                    home_game_system = line.select('td')[7].get_text()
                    guest_game_system = ''
                else:
                    home_coach = ''
                    guest_coach = line.select('td')[8].get_text()
                    home_game_system = ''
                    guest_game_system = line.select('td')[7].get_text()
                    
                try:
                    viewers = int(line.select('td')[9].get_text().replace('.', ''))
                except:
                    viewers = 0
                result = line.select('td')[10].get_text().strip()

                hash = hashlib.sha256()
                hash.update(bytes(str(viewers), 'utf-8'))
                hash.update(bytes(str(result), 'utf-8'))
                hash.update(bytes(str(date), 'utf-8'))
                hash.update(bytes(str(time), 'utf-8'))

                game_list.append({
                    'saison': saison,
                    'competition': competition,
                    'gameday': gameday,
                    'date': date,
                    'time': time,
                    'home_team_id': home_team_id,
                    'home_team_name': home_team_name,
                    'guest_team_id': guest_team_id,
                    'guest_team_name': guest_team_name,
                    'home_game_system': home_game_system,
                    'guest_game_system': guest_game_system,
                    'home_coach': home_coach,
                    'guest_coach': guest_coach,
                    'viewers': viewers,
                    'result': result,
                    "identifier": hash.hexdigest()
                })

                #print(game_list[-1])

2018


In [9]:
game_list[0], len(game_list)

({'saison': '2018/19',
  'competition': 'Bundesliga',
  'gameday': '1',
  'date': 'Fr. 27.07.2018',
  'time': '20:45',
  'home_team_id': '14',
  'home_team_name': 'FK Austria Wien',
  'guest_team_id': '2444',
  'guest_team_name': 'FC Wacker Innsbruck',
  'home_game_system': '',
  'guest_game_system': '5-4-1 flach',
  'home_coach': '',
  'guest_coach': 'Karl Daxbacher',
  'viewers': 13155,
  'result': '2:1',
  'identifier': '9e2ade517af5d236a8cd40654c0b8f38363e4d7b191c0ad767a547febf627496'},
 469)

In [10]:
df = pd.DataFrame(game_list)

In [11]:
df.head()

,competition,date,gameday,guest_coach,guest_game_system,guest_team_id,guest_team_name,home_coach,home_game_system,home_team_id,home_team_name,identifier,result,saison,time,viewers
0,Bundesliga,Fr. 27.07.2018,1,Karl Daxbacher,5-4-1 flach,2444,FC Wacker Innsbruck,,,14,FK Austria Wien,9e2ade517af5d236a8cd40654c0b8f38363e4d7b191c0a...,2:1,2018/19,20:45,13155
1,Bundesliga,Sa. 04.08.2018,2,,,122,SK Sturm Graz,Karl Daxbacher,4-4-2 doppel 6,2444,FC Wacker Innsbruck,c8db4c597ca5d0abfb449ef80cc75efbffbb9122afbca5...,2:3,2018/19,17:00,7820
2,Bundesliga,Sa. 11.08.2018,3,Karl Daxbacher,4-2-3-1,2444,FC Wacker Innsbruck,,,3551,SC Rheindorf Altach,d5ff103daeaae2db7548a6cfe4ab826dca4fab8b9b4942...,1:2,2018/19,17:00,5845
3,Bundesliga,Sa. 18.08.2018,4,,,2662,SKN St. Pölten,Karl Daxbacher,4-4-2 doppel 6,2444,FC Wacker Innsbruck,7667ba116b5e2e23eafab6f8ec6546d478144f57d91636...,0:2,2018/19,17:00,4321
4,Bundesliga,So. 26.08.2018,5,Karl Daxbacher,4-2-3-1,2444,FC Wacker Innsbruck,,,170,SK Rapid Wien,0c2b28d8eb7418492d1bbdd7c2ed170d8986db97b42bfa...,2:1,2018/19,17:00,17400


In [12]:
df.tail()

,competition,date,gameday,guest_coach,guest_game_system,guest_team_id,guest_team_name,home_coach,home_game_system,home_team_id,home_team_name,identifier,result,saison,time,viewers
464,Bundesliga,Sa. 04.05.2019,29,,,170,SK Rapid Wien,Reiner Geyer,?,503,FC Admira Wacker Mödling,fef1673a2388c7918b18c30cbbb6c37a974209530f73c7...,3:4,2018/19,17:00,3800
465,Bundesliga,Sa. 11.05.2019,30,Reiner Geyer,?,503,FC Admira Wacker Mödling,,,3551,SC Rheindorf Altach,58f6bb1865ad33ac9a2760a68596e5f32e2e0b399e19ba...,2:2,2018/19,17:00,3508
466,Bundesliga,Sa. 18.05.2019,31,,,2444,FC Wacker Innsbruck,Reiner Geyer,3-5-2,503,FC Admira Wacker Mödling,4e3ca0a8bd89e2c671d8b835f6b8c230768db81fef2461...,3:2,2018/19,17:00,3850
467,Bundesliga,Sa. 25.05.2019,32,Reiner Geyer,4-3-3 defensiv,503,FC Admira Wacker Mödling,,,4467,TSV Hartberg,88c8e4e9126d2ea1fa47d9763aca00f66a1132aeebd7fa...,3:1,2018/19,17:00,3873
468,ÖFB-Cup,Fr. 20.07.2018,1.Runde,Ernst Baumeister,4-5-1,503,FC Admira Wacker Mödling,,,9114,SC Neusiedl/See,d97a92f564e7dffff4add58ab4988ec38b839c4048b46a...,1:0,2018/19,19:00,800


In [13]:
df[df.result.str.len() > 3]

,competition,date,gameday,guest_coach,guest_game_system,guest_team_id,guest_team_name,home_coach,home_game_system,home_team_id,home_team_name,identifier,result,saison,time,viewers
68,ÖFB-Cup,Mi. 26.09.2018,2.Runde,,,170,SK Rapid Wien,Klaus Schmidt,5-4-1 flach,856,SV Mattersburg,f8afc2ee71b4187917a1c32ce26982f270d281c808a67e...,4:5 n.E.,2018/19,18:30,5300
162,ÖFB-Cup,Mi. 03.04.2019,Halbfinale,,,170,SK Rapid Wien,Oliver Glasner,3-4-3 flach,413,LASK,509eb7bbafe1c59b482ea39a81de9762fa72542f60b130...,4:5 n.E.,2018/19,20:30,6087
195,ÖFB-Cup,Fr. 20.07.2018,1.Runde,Markus Schopp,4-4-2 doppel 6,4467,TSV Hartberg,,,10131,SV Grödig,1c77f8e31f5e3d2ef092d1ba7eccbeaf6f885923ba1fa4...,1:3 n.V.,2018/19,18:30,150
426,ÖFB-Cup,Mi. 26.09.2018,2.Runde,Goran Djuricin,4-2-3-1,170,SK Rapid Wien,,,856,SV Mattersburg,f8afc2ee71b4187917a1c32ce26982f270d281c808a67e...,4:5 n.E.,2018/19,18:30,5300
429,ÖFB-Cup,Mi. 03.04.2019,Halbfinale,Dietmar Kühbauer,4-2-3-1,170,SK Rapid Wien,,,413,LASK,509eb7bbafe1c59b482ea39a81de9762fa72542f60b130...,4:5 n.E.,2018/19,20:30,6087


In [14]:
league = page_league.split('/')[3]
season_start = min(season_list)
season_end = max(season_list)
file_name = league + '-' + str(season_start) + '-' + str(season_end)
file_name

'bundesliga-2018-2018'

In [15]:
df.to_csv(file_name + '.csv')

In [16]:
with open(file_name + '.json', 'w', encoding='utf-8') as f:
    json.dump(game_list, f, ensure_ascii=False, indent=4)